In [1]:
import keras
from keras.models import Sequential
from keras.layers import GRU, Dense, TimeDistributed

In [3]:
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.preprocessing import image
from keras.utils import to_categorical

def process_frame(frame, previous_frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Compute the absolute difference between current and previous frame
    if previous_frame is not None:
        diff = cv2.absdiff(blurred, previous_frame)
    else:
        diff = blurred

    # Apply thresholding to emphasize moving regions
    _, threshold = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)

    # Find contours of moving objects
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around moving objects
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Update the previous frame
    previous_frame = blurred.copy()

    return frame, previous_frame


# Define the dataset path
dataset_path = '/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET'

# Get the list of labels from the subfolder names
labels = sorted(os.listdir(dataset_path))

# Create an empty list to store the frames and labels
frames = []
labels_encoded = []

# Iterate through the subfolders
for label_idx, label in enumerate(labels):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):
        # Iterate through the video files in each subfolder
        for video_file in os.listdir(label_path):
            video_path = os.path.join(label_path, video_file)
            if video_path.endswith('.avi') or video_path.endswith('.mp4'):
                # Process each video frame by frame
                cap = cv2.VideoCapture(video_path)
                previous_frame = None
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    processed_frame, previous_frame = process_frame(frame, previous_frame)
                    frames.append(processed_frame)
                    labels_encoded.append(label_idx)
                cap.release()

# Convert the frames and labels to numpy arrays
frames = np.array(frames)
labels_encoded = np.array(labels_encoded)

# Convert the labels to one-hot encoding
labels_onehot = to_categorical(labels_encoded)

# Split the data into training and testing sets
split_ratio = 0.8
split_idx = int(len(frames) * split_ratio)

train_frames = frames[:split_idx]
train_labels = labels_onehot[:split_idx]

test_frames = frames[split_idx:]
test_labels = labels_onehot[split_idx:]

# Define the RNN model
model = Sequential()
model.add(TimeDistributed(LSTM(128, return_sequences=True), input_shape=(None, frame_shape)))
model.add(TimeDistributed(LSTM(128)))
model.add(Dense(len(labels), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_frames, train_labels, epochs=10, batch_size=16, validation_data=(test_frames, test_labels))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_frames, test_labels)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


In [1]:
import os
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.preprocessing import image
from keras.utils import to_categorical

In [2]:
def process_frame(frame, previous_frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    if previous_frame is not None:
        diff = cv2.absdiff(blurred, previous_frame)
    else:
        diff = blurred
    _, threshold = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    previous_frame = blurred.copy()
    return frame, previous_frame

In [3]:
dataset_path = '/Users/vedantpadole/Desktop/Research/ASL_MY_DATASET'
labels = sorted(os.listdir(dataset_path))
frames = []
labels_encoded = []
for label_idx, label in enumerate(labels):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):
        for video_file in os.listdir(label_path):
            video_path = os.path.join(label_path, video_file)
            if video_path.endswith('.avi') or video_path.endswith('.mp4'):
                cap = cv2.VideoCapture(video_path)
                previous_frame = None
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    processed_frame, previous_frame = process_frame(frame, previous_frame)
                    frames.append(processed_frame)
                    labels_encoded.append(label_idx)
                cap.release()

In [4]:
frames = np.array(frames)
labels_encoded = np.array(labels_encoded)
labels_onehot = to_categorical(labels_encoded)


split_ratio = 0.8
split_idx = int(len(frames) * split_ratio)

train_frames = frames[:split_idx]
train_labels = labels_onehot[:split_idx]

test_frames = frames[split_idx:]
test_labels = labels_onehot[split_idx:]


frame_shape = train_frames.shape[1:]


<ipython-input-4-2dcfd2ff8cbf>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  frames = np.array(frames)


In [14]:
frame_shape = train_frames.shape[0]
frame_shape

74902

In [15]:
num_frames, num_rows, num_cols, num_channels = frames.shape
train_frames = train_frames.reshape(num_frames, num_rows * num_cols * num_channels)
test_frames = test_frames.reshape(len(test_frames), num_rows * num_cols * num_channels)

ValueError: not enough values to unpack (expected 4, got 1)

In [18]:
frames = np.array(frames)
labels_encoded = np.array(labels_encoded)

# Convert the labels to one-hot encoding
labels_onehot = to_categorical(labels_encoded)

# Split the data into training and testing sets
split_ratio = 0.8
split_idx = int(len(frames) * split_ratio)

train_frames = frames[:split_idx]
train_labels = labels_onehot[:split_idx]

test_frames = frames[split_idx:]
test_labels = labels_onehot[split_idx:]

# Reshape the frames to have consistent dimensions
num_frames= frames.shape
train_frames = train_frames.reshape(num_frames, num_rows, num_cols, num_channels)
test_frames = test_frames.reshape(len(test_frames), num_rows, num_cols, num_channels)

# Define the RNN model

NameError: name 'num_rows' is not defined